In [76]:
import tensorflow as tf
import tflearn
import numpy as np
tf.reset_default_graph() # THIS IS NECESSARY BEFORE MAKING NEW SESSION TO STOP IT ERRORING!!
try:
    sess
except:
    pass
else:
    sess.close()
    del sess
sess = tf.InteractiveSession()

n_inputs = 4  # == env.observation_space.shape[0]
n_hidden = 20  # it's a simple task, we don't need more than this
n_outputs = 2 # only outputs the probability of accelerating left
initializer = tf.contrib.layers.variance_scaling_initializer()
LR = 1e-2

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape=[None, n_inputs])
Y = tf.placeholder(tf.int32, shape=[None])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.relu,
                         kernel_initializer=initializer)
hidden1 = tf.layers.dense(hidden, n_hidden, activation=tf.nn.relu,
                         kernel_initializer=initializer)

logits = tf.layers.dense(hidden1, n_outputs,
                          kernel_initializer=initializer)
outputs = tf.nn.softmax(logits)
# 3. Select a random action based on the estimated probabilities
# p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(outputs), num_samples=1)[0]

loss = tf.losses.sparse_softmax_cross_entropy(labels=Y, logits=logits)
train_step = tf.train.AdamOptimizer(LR).minimize(loss)
    
tf.global_variables_initializer().run()
saver = tf.train.Saver()

# a = np.expand_dims(np.random.rand(4),axis=0)
# print(sess.run([logits,outputs,action], feed_dict={X: a}))


//anaconda/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [82]:
import gym
# env = gym.make('CartPole-v0')
env = gym.make("CartPole-v0").env
i = 0
rewards_t = []
    
while i < 50:
    observation = env.reset()
    acts = []
    obvs = []
    reward_t = 0
    for t in range(1000):
#         env.render()
        act = sess.run(action, feed_dict={X: np.expand_dims(observation,axis=0)})[0]
        acts.append(act)
        obvs.append(observation)
        observation, reward, done, info = env.step(act)
        reward_t += reward
        if done:
            t_s.append(t)
            rewards_t.append(reward_t)
            if (reward_t > (np.mean(rewards_t[-10:-1]) + 0.5 * np.std(rewards_t[-10:-1]))):
                print('training! %d' % reward_t)
                sess.run(train_step, feed_dict={X: obvs, Y:acts})
                max_t = t
                i += 1
            break
#     if np.mean(rewards_t[-10:-1]) > 180:
#        break
            
saver.save(sess, "./my_policy_net_basic.ckpt")

env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
//anaconda/envs/tensorflow/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


training! 756
training! 930
training! 644
training! 656
training! 569
training! 704
training! 491
training! 519
training! 975
training! 735
training! 693
training! 748
training! 529
training! 720
training! 556
training! 597
training! 766
training! 546
training! 907
training! 679
training! 886
training! 515
training! 808
training! 714
training! 582
training! 688
training! 753
training! 924
training! 562
training! 617
training! 463
training! 541
training! 475
training! 532
training! 549
training! 742
training! 656
training! 693
training! 790
training! 645
training! 923
training! 731
training! 781
training! 611
training! 588
training! 721
training! 736
training! 641
training! 646
training! 704


In [83]:
frames = []
rewardsum = 0
env = gym.make("CartPole-v0").env
obs = env.reset()
with tf.Session() as sess:
    saver.restore(sess, "./my_policy_net_basic.ckpt")
    for step in range(1000):
        env.render(obs)
        action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})
        obs, reward, done, info = env.step(action_val[0])
        rewardsum += reward
        if done:
            break
env.close()
print(rewardsum)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from ./my_policy_net_basic.ckpt


//anaconda/envs/tensorflow/lib/python3.6/site-packages/gym/envs/classic_control/cartpole.py:137: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self.viewer.render(return_rgb_array = mode=='rgb_array')


363.0
